In [1]:
import pandas as pd #1
import datetime     #2
import os           #3

In [2]:
given_data_folder = 'data'

file_names = os.listdir(given_data_folder)
file_names.sort()
file_names

['MRegularSeasonDetailedResults.csv', 'bracket-2022.csv']

In [3]:
file_name = 'MRegularSeasonDetailedResults.csv'

In [4]:
file_path = given_data_folder + '/' + file_name 
given_df  = pd.read_csv(file_path)
print(len(given_df))

given_df.head(5)

100423


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


## get only games from 2022 season

In [5]:
given_df = given_df[given_df.Season == 2022]
print(len(given_df))

given_df.head(5)

3736


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
96687,2022,8,1104,93,1256,64,H,0,32,64,...,26,6,19,14,20,11,10,3,1,18
96688,2022,8,1105,82,1398,73,H,0,31,71,...,27,2,5,14,27,12,14,5,2,16
96689,2022,8,1110,77,1265,73,H,1,27,59,...,19,7,12,10,26,5,15,6,3,20
96690,2022,8,1112,81,1319,52,H,0,28,69,...,16,9,17,11,28,11,20,3,1,22
96691,2022,8,1113,76,1339,60,H,0,31,66,...,24,10,11,2,22,10,21,9,1,14


## change from df to just a list of columns

In [6]:
# Creates list of the column names from the df
columns = given_df.columns.tolist()

# Creates list of the columns
df_columns = []
for expected_column in columns:
    temp = given_df[expected_column].tolist()
    df_columns.append(temp)
    
len(df_columns)

34

In [7]:
# Make a list of rows too
df_rows = []

for i in range(len(df_columns[0])):
    new_row = []
    
    for j in range(len(df_columns)):
        new_row.append(df_columns[j][i])
    
    df_rows.append(new_row)

len(df_rows)

3736

In [8]:
row   = ""
count = 0
for i in range(len(columns)):
    temp   = "{:>3}: {}".format(i, columns[i])
    row   += "{:<15}"   .format(temp)
    count += 1
    
    if count > 6:
        row   +=("\n")
        count  = 0

print(row)

  0: Season      1: DayNum      2: WTeamID     3: WScore      4: LTeamID     5: LScore      6: WLoc      
  7: NumOT       8: WFGM        9: WFGA       10: WFGM3      11: WFGA3      12: WFTM       13: WFTA      
 14: WOR        15: WDR        16: WAst       17: WTO        18: WStl       19: WBlk       20: WPF       
 21: LFGM       22: LFGA       23: LFGM3      24: LFGA3      25: LFTM       26: LFTA       27: LOR       
 28: LDR        29: LAst       30: LTO        31: LStl       32: LBlk       33: LPF       


## get each teams stats

In [9]:
game_solutions = []
# team1 ID, team2 ID, team1 win? (1 or 0)

team_dictionary = {}
# key = teamID
# value = [count, team]

for i in range(len(df_rows)):
    row = df_rows[i]
    
    # game_solutions
    # ---------------
    game_solutions.append( [row[2], row[4], 1] )
    
    
    # team dictionary
    # ----------------
    
    #team_for     = [row[3]] + row[ 8:21].copy()
    #team_allowed = [row[5]] + row[21:  ].copy()
    
    team_for     = row[ 8:21].copy()
    team_allowed = row[21:  ].copy()
    
    
    # WTEAM
    if row[2] not in team_dictionary:
        team_dictionary[row[2]] = [1, team_for, team_allowed]
    else:
        # count
        team_dictionary[row[2]][0] += 1
        # team for
        for j in range(len(team_for)):
            team_dictionary[row[2]][1][j] += team_for[j]
            team_dictionary[row[2]][2][j] += team_allowed[j]

    # LTEAM
    if row[4] not in team_dictionary:
        team_dictionary[row[4]] = [1, team_allowed, team_for]
    else:
        # count
        team_dictionary[row[4]][0] += 1
        # team for
        for j in range(len(team_for)):
            team_dictionary[row[4]][1][j] += team_allowed[j]
            team_dictionary[row[4]][2][j] += team_for[j]


            
            
# turn (defensive rebounds) and (offensive rebounds) into (average total rebound differential)
# ---------------------------------------------------------------------------------------------

for i in range(len( list(team_dictionary.keys()) )):
    row = team_dictionary[ list(team_dictionary.keys())[i] ]
    # row = [team_id, [stats_for], [stats_allowed]]
    
    # dont do this twice
    # -------------------
    if len(row[1]) > 12: 
    
        # stats:
        # 0: FGM, 1: FGA, 2: FGM3, 3: FGA3, 4: FTM, 5: FTA, 6: OR, 7: DR, 8: Ast, 9: TO, 10: Stl 11: Blk, 12: PF
        total_rebounds_for     = (row[1][6] + row[1][7])
        total_rebounds_against = (row[2][6] + row[2][7])

        rebound_differential_for     = (total_rebounds_for - total_rebounds_against)
        rebound_differential_against = (total_rebounds_against - total_rebounds_for)

        # pop defensive rebounds
        row[1].pop(7)
        row[2].pop(7)

        # replace offensive rebounds with rebound differential
        row[1][6] = rebound_differential_for
        row[2][6] = rebound_differential_against

        # new_stats:
        # 0: FGM, 1: FGA, 2: FGM3, 3: FGA3, 4: FTM, 5: FTA, 6: RDif, 7: Ast, 8: TO, 9: Stl 10: Blk, 11: PF

    
print("0: FGM, 1: FGA, 2: FGM3, 3: FGA3, 4: FTM, 5: FTA, 6: RDif, 7: Ast, 8: TO, 9: Stl 10: Blk, 11: PF")


0: FGM, 1: FGA, 2: FGM3, 3: FGA3, 4: FTM, 5: FTA, 6: RDif, 7: Ast, 8: TO, 9: Stl 10: Blk, 11: PF


In [10]:
# NEW - USING CUSTOM REBOUND DATA

# print report on the collected stats
# ------------------------------------
print("game_solutions  length: ", len(game_solutions ))
print("team_dictionary length: ", len(team_dictionary), "\n")

# print example data
# -------------------
print("  id    FGM   FGA  FGM3  FGA3   FTM   FTA  RDif   Ast    TO   Stl   Blk    PF")
#        23: [1110, 2585,  355, 1112,  573,  797,  170,              560,  282,  164,  746] - 11 values

#print(" id    FGM   FGA  FGM3  FGA3   FTM   FTA    OR    DR   Ast    TO   Stl   Blk    PF")
#        23: [1110, 2585,  355, 1112,  573,  797,  448, 1028,  568,  560,  282,  164,  746] - 13 values

for i in range(1):
    line = "{:>4}: [".format( team_dictionary[list(team_dictionary.keys())[i]][0] )
    
    for stat in team_dictionary[list(team_dictionary.keys())[i]][1]:
        line += "{:>4}, ".format(stat)
    
    line = line[:-2] + "] - {} values".format(len(team_dictionary[list(team_dictionary.keys())[i]][1]))
    print(line, "\n")
    

game_solutions  length:  3736
team_dictionary length:  358 

  id    FGM   FGA  FGM3  FGA3   FTM   FTA  RDif   Ast    TO   Stl   Blk    PF
  23: [1110, 2585,  355, 1112,  573,  797,   76,  568,  560,  282,  164,  746] - 12 values 



In [11]:
# OLD - USING BASE REBOUND DATA
'''
# print report on the collected stats
# ------------------------------------
print("game_solutions  length: ", len(game_solutions ))
print("team_dictionary length: ", len(team_dictionary), "\n")

# print example data
# -------------------
print("  id    FGM   FGA  FGM3  FGA3   FTM   FTA    OR    DR   Ast    TO   Stl   Blk    PF")
#        23: [1110, 2585,  355, 1112,  573,  797,  448, 1028,  568,  560,  282,  164,  746] - 13 values

for i in range(1):
    line = "{:>4}: [".format( team_dictionary[list(team_dictionary.keys())[i]][0] )
    
    for stat in team_dictionary[list(team_dictionary.keys())[i]][1]:
        line += "{:>4}, ".format(stat)
    
    line = line[:-2] + "] - {} values".format(len(team_dictionary[list(team_dictionary.keys())[i]][1]))
    print(line, "\n")
    
''';

## change to averages

In [12]:
team_averages = {}
team_keys = list(team_dictionary.keys())

for key in team_keys:
    entry = team_dictionary[key]
    count = entry[0]
    
    avg_team_for     = []
    avg_team_allowed = []
    
    for i in range(len(entry[1])):
        avg_team_for.append( (entry[1][i] / count) )
        avg_team_allowed.append( (entry[2][i] / count) )
        
    team_averages[key] = [avg_team_for, avg_team_allowed]
    
print(len(team_averages), "\n")
#for i in range(2):
    #print(team_averages[ list(team_averages.keys())[i] ])
    #print()

358 



# Testing With The Network

In [13]:
import sys
sys.path.append("..")

In [14]:
import random
import time
import threading
import math

In [15]:
from network_classes.neuron      import Neuron
from network_classes.agent       import Agent
from network_classes.population  import Population

from extra.progress_package import *

## setting up inputs and solutions

In [16]:
inputs = []
solutions = []

for i in range(len(game_solutions)):
    game = game_solutions[i]
    team1 = game[0]
    team2 = game[1]
    
    if random.uniform(0,1) > 0.5:
        #input_row = team_averages[team1][1] + team_averages[team2][1]
        input_row = []
        for j in range(len(team_averages[team1][0])):
            input_row.append( (team_averages[team1][0][j] - team_averages[team2][0][j]) )
        
        inputs.append(input_row)
        solutions.append(1)
        
        # second way around
        input_row = []
        for j in range(len(team_averages[team1][0])):
            input_row.append( (team_averages[team2][0][j] - team_averages[team1][0][j]) )
        
        inputs.append(input_row)
        solutions.append(0)
        
        
        
    else:
        #input_row = team_averages[team2][1] + team_averages[team1][1]
        input_row = []
        for j in range(len(team_averages[team1][0])):
            input_row.append( (team_averages[team2][0][j] - team_averages[team1][0][j]) )
        
        inputs.append(input_row)
        solutions.append(0)
        
        
        # second way around
        input_row = []
        for j in range(len(team_averages[team1][0])):
            input_row.append( (team_averages[team1][0][j] - team_averages[team2][0][j]) )
        
        inputs.append(input_row)
        solutions.append(1)
        
        
        
        
print("inputs length:", len(inputs))
print("solutions length:", len(solutions))

inputs length: 7472
solutions length: 7472


In [17]:
# training ->   0:2615
# testing -> 2615:

In [18]:
split_count = 10

inputs_splits = []
solutions_splits = []

for i in range(split_count):
    actual_length = len(inputs) // 2
    split_length  = (actual_length // split_count) * 2
    
    start_index =  i    * split_length
    end_index   = (i+1) * split_length
    
    inputs_splits.append(    inputs   [start_index:end_index].copy() )
    solutions_splits.append( solutions[start_index:end_index].copy() )
    
    #print(len( inputs[start_index:end_index] ))


## First Test

# THREADING
# -------------------

### initialize population

In [19]:
TEST_inputs = inputs[:2615].copy()
TEST_solutions = solutions[:2615].copy()

TEST_population = Population( len(TEST_inputs[0]) , 10)

#for i in range(10):
    #for agent in TEST_population.agents:
        #agent.mutate()

TEST_population.test_agents_double(inputs, solutions)

### variables for stat tracking

In [20]:
# total values
# -------------
total_timer = 0
total_steps = 0

total_steps_sec = 0
total_secs_step = 0

total_evolution_timer = 0
total_scoring_timer   = 0


# previous values
# ----------------
prev_timer     = 0
prev_steps     = 0

prev_steps_sec = 0
prev_secs_step = 0

prev_evolution_timer = 0
prev_scoring_timer   = 0

prev_max_conf  = 0
prev_max_base  = 0
prev_accuracy  = 0


# Training Loop

In [ ]:
# training time controls
# -----------------------
training_time = 240 # (in seconds)
max_steps     = 3600 # (overall limit just in case)


# for stat tracking
# ------------------
timer = 0
steps = 0

evolution_timer = 0
scoring_timer   = 0


# stats for loop control
# -----------------------
start_time = time.time()
elapsed    = 0

# loop start
# -----------
for i in range(max_steps):
    elapsed = time.time() - start_time
    
    # check if loop should be done
    # -----------------------------
    if elapsed > training_time: 
        progress_bar(training_time, training_time)
        break
    else: 
        # show progress update
        progress_bar(elapsed, training_time)
        pass
    
    
    # evolution
    # ----------
    evolution_start = time.time()
    TEST_population.evolution_step()
    evolution_timer += (time.time() - evolution_start)

    # scoring
    # --------
    scoring_start = time.time()
    TEST_population.test_agents_threading_double(inputs, solutions) # (inputs_splits[i%10], solutions_splits[i%10])
    scoring_timer += (time.time() - scoring_start)
    
    
    steps += 1


timer += (time.time() - start_time)
    
# check scores
# -------------
TEST_population.evolution_step(reset_confidence_scores=False)
steps += 1
TEST_population.test_agents_double(inputs, solutions)


# base scores
base_scores   = TEST_population.base_scores.copy()
unsorted_base = base_scores.copy()
base_scores.sort(reverse=True)

# conf_scores
conf_scores   = TEST_population.scores.copy()
unsorted_conf = conf_scores.copy()
conf_scores.sort(reverse=True)



# calculate some values initially
# --------------------------------

# current values
# ---------------
steps_sec = round((steps/timer), 2)
secs_step = round((timer/steps), 2)

max_conf = max(unsorted_conf)
max_base = max(unsorted_base)
accuracy = round(( max_base/(len(inputs)//2) )*100, 2)


# total values
# -------------
total_timer += timer
total_steps += steps

total_steps_sec = round((total_steps/total_timer), 2)
total_secs_step = round((total_timer/total_steps), 2)

total_evolution_timer += evolution_timer
total_scoring_timer   += scoring_timer


# calculate stats for lines
# --------------------------
#                     new_total,          prev_total,                   change
timer_l = [round(total_timer,2), round(prev_timer,2),           round(timer,2)]
steps_l = [         total_steps,          prev_steps,                    steps]
steps_s = [     total_steps_sec,      prev_steps_sec,                steps_sec]
s_steps = [     total_secs_step,      prev_secs_step,                secs_step]
base_l  = [            max_base,       prev_max_base, (max_base-prev_max_base)]
acc_l   = [            accuracy,       prev_accuracy, (accuracy-prev_accuracy)] 

conf_l  = [      round(max_conf), round(prev_max_conf), round(max_conf-prev_max_conf)]
evo_t   = [round(total_evolution_timer,2), round(prev_evolution_timer,2), round(evolution_timer,2)]
score_t = [  round(total_scoring_timer,2),   round(prev_scoring_timer,2),   round(scoring_timer,2)]


# format lines and print them
# ----------------------------
ln1  = "            current    prev       change  "
ln2  = "            --------   --------   --------"
ln3  = "timer      {:-8.2f}s  {:-8.2f}s  {:-8.2f}s".format(timer_l[0], timer_l[1], timer_l[2])
ln4  = "steps      {:-8}   {:-8}   {:-8} ".format(steps_l[0], steps_l[1], steps_l[2])
#ln5  = "steps/sec  {:-8.2f}   {:-8.2f}   {:-8.2f} ".format(steps_s[0], steps_s[1], steps_s[2])
ln6  = "secs/step  {:-8.2f}s  {:-8.2f}s  {:-8.2f}s".format(s_steps[0], s_steps[1], s_steps[2])
ln7  = "evolu time {:-8.2f}s  {:-8.2f}s  {:-8.2f}s".format(evo_t[0], evo_t[1], evo_t[2])
ln8  = "score time {:-8.2f}s  {:-8.2f}s  {:-8.2f}s".format(score_t[0], score_t[1], score_t[2])
ln9  = "            --------   --------   --------"
ln10 = "conf score {:-8}   {:-8}   {:-8} ".format(conf_l[0], conf_l[1], conf_l[2])
ln11 = "base score {:-8}   {:-8}   {:-8} ".format(base_l[0], base_l[1], base_l[2])
ln12 = "accuracy   {:-8.2f}%  {:-8.2f}%  {:-8.2f}%".format(acc_l[0], acc_l[1], acc_l[2])

all_lines = ["\n", ln1, ln2, ln3, ln4, ln6, ln7, ln8, ln9, ln10, ln11, ln12]
for stat_line in all_lines:
    print(stat_line)


# set previous variables
# -----------------------
prev_timer     = total_timer
prev_steps     = total_steps

prev_steps_sec = total_steps_sec
prev_secs_step = total_secs_step

prev_evolution_timer = total_evolution_timer
prev_scoring_timer   = total_scoring_timer

prev_max_conf  = max_conf 
prev_max_base  = max_base
prev_accuracy  = accuracy




# print out final scores
# -----------------------
print()
print("id - [ c r, nc r, nc w,  c w] | ovrl |  conf  ({})".format( round(prev_max_conf) ))
print("---------------------------------------------")
#       0 - [2030,  209,  202, 1295] | 2974 | 2239

for i in range(len(TEST_population.agents)):
    # get the confidence scores
    conf = TEST_population.confidence_scores[i]
    conf_scores_list = "[{:>4}, {:>4}, {:>4}, {:>4}]".format(conf[0], conf[1], conf[2], conf[3])
    
    print( " {} - {} | {:>4} | {:>5}".format(i, conf_scores_list, unsorted_base[i], round(unsorted_conf[i])) )
    

In [ ]:
deletion_time = 0
for i in range(len(TEST_population.agents)):
    deletion_time += TEST_population.agents[i].time_deleting
    
deletion_time

In [23]:
max_score = 0
max_index = 0
for i in range(len(TEST_population.agents)):
    if TEST_population.scores[i] > max_score:
        max_score = TEST_population.scores[i]
        max_index = i
        
TEST_population.agents[max_index].print1()

format:

 # neurons    |  final value        
 ------------ |  --------------     
 average from |  (# to, # from) 


          FGM    FGA   FGM3   FGA3    FTM    FTA   RDif    Ast     TO    Stl    Blk     PF
  12 |   3.25  -4.71   2.62   1.00  -5.79  -8.50   1.38   2.62  -0.29  -0.08  -0.58  -3.67 
 4.1 |  (0,4)  (0,4)  (0,5)  (0,4)  (0,3)  (0,4)  (0,6)  (0,4)  (0,4)  (0,4)  (0,5)  (0,2) 
      
  12 |   1.00  -1.00  -1.00  -0.62  -0.62   0.28  -1.00   1.00  -1.00   0.93   0.70   1.00 
 4.0 |  (5,5)  (4,4)  (6,5)  (2,5)  (5,2)  (2,2)  (2,6)  (5,4)  (8,5)  (6,2)  (3,3)  (1,5) 
      
  12 |   1.00  -0.21  -0.78  -0.85  -0.54   0.70   0.50  -0.73   0.78  -0.04   0.52  -0.27 
 2.8 |  (4,3)  (4,4)  (3,3)  (6,3)  (6,3)  (6,1)  (4,5)  (4,2)  (3,3)  (4,2)  (2,2)  (2,3) 
      
  10 |   0.78   0.68   0.57   0.35   0.47   0.66  -0.36  -0.07  -0.42  -0.70 
 1.0 |  (2,1)  (8,1)  (2,1)  (2,1)  (2,1)  (3,1)  (3,1)  (3,1)  (4,1)  (5,1) 
      
   1 |   0.91 
 0.0 | (10,0) 
      

5 layers, 47 neur

In [24]:
print(len(solutions))
solutions[-1]

7472


1

In [26]:
#3220*7472
# 1610*7472 = 12_029_920
# 3220*7472 = 24_059_840

24059840

In [25]:
randint_time = 0
for i in range(10000):
    start_time = time.time()
    random.randint(0, 1000)
    randint_time += time.time() - start_time

random_time = 0
for i in range(10000):
    start_time = time.time()
    math.floor(1000*random.random())
    random_time += time.time() - start_time
    
uniform_time = 0
for i in range(10000):
    start_time = time.time()
    random.uniform(0,1)
    uniform_time += time.time() - start_time
    
    
print( "random.randint(a,b) : {}s".format(randint_time ) )
print( "random.random()     : {}s".format(random_time  ) )
print( "random.uniform()    : {}s".format(uniform_time ) )

random.randint(a,b) : 0.014762401580810547s
random.random()     : 0.003671407699584961s
random.uniform()    : 0.0042247772216796875s
